In [1]:
import os
from dotenv import load_dotenv
import google.generativeai as genai
from langchain_google_genai import GoogleGenerativeAIEmbeddings


/home/ashutosh/Desktop/GenAI/LLMOops/rag-lloops/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data Injestion

In [2]:
from langchain.document_loaders import TextLoader

In [3]:
loader = TextLoader("/home/ashutosh/Desktop/GenAI/LLMOops/rag-lloops/data/agentic-ai.txt", encoding='utf8')
documents=loader.load()

In [4]:
documents[0]

Document(metadata={'source': '/home/ashutosh/Desktop/GenAI/LLMOops/rag-lloops/data/agentic-ai.txt'}, page_content='Agentic AI: The Evolution of Autonomous Intelligence\n\nIntroduction\n\nArtificial Intelligence (AI) has undergone a remarkable transformation in recent years, evolving from simple rule-based systems to sophisticated models capable of complex reasoning and decision-making. Among the most exciting developments in this field is the emergence of Agentic AI, a paradigm where AI systems exhibit autonomous, goal-directed behavior, acting as independent agents capable of perceiving their environment, making decisions, and executing actions to achieve specific objectives. Unlike traditional AI systems that rely heavily on human input or predefined scripts, agentic AI systems possess a degree of autonomy, adaptability, and proactivity, enabling them to operate in dynamic and unpredictable environments.\n\nThis essay explores the concept of agentic AI, delving into its definition, c

In [5]:
documents[0].page_content[:500]

'Agentic AI: The Evolution of Autonomous Intelligence\n\nIntroduction\n\nArtificial Intelligence (AI) has undergone a remarkable transformation in recent years, evolving from simple rule-based systems to sophisticated models capable of complex reasoning and decision-making. Among the most exciting developments in this field is the emergence of Agentic AI, a paradigm where AI systems exhibit autonomous, goal-directed behavior, acting as independent agents capable of perceiving their environment, makin'

# Chucking

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,  
    chunk_overlap=20  
)

In [8]:
text_chunks=text_splitter.split_documents(documents)

In [9]:
text_chunks

[Document(metadata={'source': '/home/ashutosh/Desktop/GenAI/LLMOops/rag-lloops/data/agentic-ai.txt'}, page_content='Agentic AI: The Evolution of Autonomous Intelligence\n\nIntroduction'),
 Document(metadata={'source': '/home/ashutosh/Desktop/GenAI/LLMOops/rag-lloops/data/agentic-ai.txt'}, page_content='Artificial Intelligence (AI) has undergone a remarkable transformation in recent years, evolving from simple rule-based systems to sophisticated models capable of complex reasoning and'),
 Document(metadata={'source': '/home/ashutosh/Desktop/GenAI/LLMOops/rag-lloops/data/agentic-ai.txt'}, page_content='reasoning and decision-making. Among the most exciting developments in this field is the emergence of Agentic AI, a paradigm where AI systems exhibit autonomous, goal-directed behavior, acting as'),
 Document(metadata={'source': '/home/ashutosh/Desktop/GenAI/LLMOops/rag-lloops/data/agentic-ai.txt'}, page_content='behavior, acting as independent agents capable of perceiving their environmen

In [10]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS

In [11]:
from dotenv import load_dotenv
import os
import google.generativeai as genai
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS

# 1. Load Environment Variables
load_dotenv()

# 2. Configure the generative AI model
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

# 3. Embedding Model Setup
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001"
)

FAISS_INDEX_PATH = "faiss_index"

if os.path.exists(FAISS_INDEX_PATH):
    # Load the existing vector store
    print("Loading existing vector store...")
    vectorstore = FAISS.load_local(FAISS_INDEX_PATH, embeddings, allow_dangerous_deserialization=True)
    print("Vector store loaded successfully!")
else:
    # Create and save the vector store if it doesn't exist
    print("Creating new vector store...")
    # Data Loading and Chunking
    loader = TextLoader("/home/ashutosh/Desktop/GenAI/LLMOops/rag-lloops/data/agentic-ai.txt", encoding='utf8')
    documents = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=2000,
        chunk_overlap=200
    )
    text_chunks = text_splitter.split_documents(documents)

    # Create Vector Store
    vectorstore = FAISS.from_documents(text_chunks, embeddings)
    vectorstore.save_local(FAISS_INDEX_PATH)
    print("Vector store created and saved successfully!")

E0000 00:00:1760421993.834559   17138 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


Creating new vector store...
Vector store created and saved successfully!


In [12]:
vectorstore

In [13]:
retriever = vectorstore.as_retriever()

In [14]:
query = "What is the key characteristics of agentic AI?"
docs = vectorstore.similarity_search(query, k=4)


for i, doc in enumerate(docs):
    print(f"documents {i+1}")
    print(doc.page_content)
    print("-"*50)

documents 1
Agentic AI: The Evolution of Autonomous Intelligence

Introduction

Artificial Intelligence (AI) has undergone a remarkable transformation in recent years, evolving from simple rule-based systems to sophisticated models capable of complex reasoning and decision-making. Among the most exciting developments in this field is the emergence of Agentic AI, a paradigm where AI systems exhibit autonomous, goal-directed behavior, acting as independent agents capable of perceiving their environment, making decisions, and executing actions to achieve specific objectives. Unlike traditional AI systems that rely heavily on human input or predefined scripts, agentic AI systems possess a degree of autonomy, adaptability, and proactivity, enabling them to operate in dynamic and unpredictable environments.

This essay explores the concept of agentic AI, delving into its definition, characteristics, underlying technologies, applications, ethical implications, and future potential. Spanning a

In [15]:
from langchain.prompts import ChatPromptTemplate

template="""
You are an assistant for Q&A tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
"""

In [16]:
prompt = ChatPromptTemplate.from_template(template)

In [17]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="\nYou are an assistant for Q&A tasks.\nUse the following pieces of retrieved context to answer the question.\nIf you don't know the answer, just say that you don't know.\nUse ten sentences maximum and keep the answer concise.\nQuestion: {question}\nContext: {context}\n"), additional_kwargs={})])

In [18]:
from langchain.schema.output_parser import StrOutputParser
output_parser = StrOutputParser()

In [19]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [34]:
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

E0000 00:00:1760422982.035771   17138 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [35]:
from langchain.schema.runnable import Runnable

class ExtractQuestion(Runnable):
    """Runnable that extracts the 'question' field from an input dict and returns it as a string."""

    def invoke(self, input, run_manager=None, **kwargs):
        # handle when input is a mapping-like object
        try:
            return input["question"]
        except Exception:
            raise TypeError("Expected input to be a dict with a 'question' key")

# Create an instance to use in the chain
input_mapper = ExtractQuestion()

# Use the same mapper for the retriever's input (so retriever receives a string)
chain = (
    {"context": input_mapper | retriever, "question": input_mapper} | prompt | model | output_parser
)


In [36]:
chain.invoke({"question":"What is the key characteristics of agentic AI?"})

'Agentic AI systems are defined by several core characteristics. They exhibit autonomy, operating independently and making decisions based on goals and environmental inputs without constant human instruction. These systems display goal-directed behavior, pursuing specific objectives over short or long terms. Adaptability is crucial, as agentic AI learns from its environment to update knowledge and strategies for new situations.\n\nFurthermore, they are proactive, anticipating future states and taking initiative rather than merely reacting to stimuli. Agentic AI interacts with its environments, utilizing sensors or data interfaces to perceive and act upon the world. Finally, these systems possess advanced reasoning and decision-making capabilities, integrating multiple data sources to make informed choices.'